### Mol Bubi és Don Republic állomások (Google-ről)
- Google Places API
    - Mol Bubi és Donkey Republic állomások nevének, illetve GPS adatainak letöltésére használom
    - https://developers.google.com/maps/documentation/places/web-service
- Nominatim OpenStreetMap API
    - a letöltött GPS adatokból ezzel az API-val keresem vissza az utcaneveket
    - (Google Geocodingot is próbáltam, de ez jobbnak bizonyult)
    - https://nominatim.openstreetmap.org/ui/reverse.html

In [ ]:
import requests
import pandas as pd
from pprint import pprint
from time import sleep

- a `collect_data` függvény picit hosszú lett, lehetne még rajta egyszerűsíteni, de ez most így működik
- a Google bonyolított a helyzeten, egy ponthoz mindegy, hogy mekkora range-et jelölünk be, maximum 60 helyet ad vissza, azt is több oldalon, ezért kellenek a next_page_token-ek, ha ezek léteznek, akkor van következő oldal

In [2]:
def collect_data(keyword, latitude, longitude, radius, api_key, df):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?keyword={keyword}&location={latitude}%2C{longitude}&radius={radius}&key={api_key}"
    response_google = requests.get(url).json()
    # első oldal
    for i, result in enumerate(response_google['results']):
        
        google = collect_gps_name(response_google, i)
        if google['name'] in df['StationName'].values:
            continue
        
        osm = collect_address(google['lati'], google['longi'])       
        
        new_row = pd.Series({'StationName':google['name'], 
                             'PostalCode':osm['postcode'], 
                             'City':osm['city'], 
                             'Street':osm['road'],
                             'District':osm['district'],
                             'Latitude':google['lati'], 
                             'Longitude':google['longi']})
        df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
    
        sleep(0.5)
    
    # további oldalak, ha vannak
    for j in range(30):        
        try:
            next = response_google['next_page_token']            
            next_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next}&key={api_key}"
            response_google = requests.get(next_url).json()
            
            for k, result in enumerate(response_google['results']):
                google = collect_gps_name(response_google, k)
                if google['name'] in df['StationName'].values:
                    continue
                osm = collect_address(google['lati'], google['longi'])       

                new_row = pd.Series({'StationName':google['name'], 
                                     'PostalCode':osm['postcode'],
                                     'City':osm['city'],
                                     'District':osm['district'],
                                     'Street':osm['road'],
                                     'Latitude':google['lati'],
                                     'Longitude':google['longi']})
                df = pd.concat([df, new_row.to_frame().T], ignore_index=True)

                sleep(0.5)

        except:
            return df    
    
    return df

- `collect_gps_name` függvény a Google API-ból begyűjti az állomások nevét és GPS koordinátáit

In [3]:
def collect_gps_name(response, i):    
    name = response['results'][i]['vicinity']    
    name = name.replace("Budapest, ", "")
    
    lati = response['results'][i]['geometry']['location']['lat']    
    longi = response['results'][i]['geometry']['location']['lng']    
    
    collected_gps_name = {'name': name, 'lati': lati, 'longi': longi}
    return collected_gps_name

- a `collect_address` az előzőleg begyűjtött GPS koordinátákból Nominatim API-val megállapítja az irányítószámot, várost, kerületet, utcanevet

In [4]:
def collect_address(lati, longi):    
    url = f"https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lati}&lon={longi}"
    response = requests.get(url).json()
    try:
        postcode = response['address']['postcode']
    except:
        postcode = None
    try:
        city = response['address']['city']
    except:
        city = None
    try:
        district = response['address']['district']
        district = district.replace(". kerület", "")
        roman_dict = {'I':1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':'9', 'X':10,
                      'XI':'11', 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'XVII':17, 'XVIII':18, 'XIX':19, 
                      'XX':20, 'XXI':21, 'XXII':22, 'XXIII':23}
        district = roman_dict[district]
    except:
        district = None
    try:
        road = response['address']['road']
    except:
        road = None
    
    collected_address = {'postcode':postcode, 'city':city, 'district':district, 'road':road} 
    return collected_address

- Pandas DataFrame-k létrehozása Mol Bubi-hoz és Donkey Republic állomásokhoz

In [5]:
mol = pd.DataFrame(columns = ["StationName", "PostalCode", "City", "District", "Street", "Latitude", "Longitude"])
don = mol

- Google Places API key beolvasása egy txt-ből

In [6]:
with open(r'C:\vremek\google_places_api_key.txt', 'r') as f:
    api_key = f.read()

- generáltam GPS koordinátákat Budapest középső részére, ezeken a pontokon vizsgálja át a "helyeket", így sikerült a legtöbb ereményt kapnom
- (47.44, 19.02) - (47.52, 19.12)    
    - a pontok egyes koordinátái mindig 1 századdal változnak

In [7]:
gps = []
for i in range(11):
    for j in range(9):
        gps.append([round(47.44 + j*0.01,2), round(19.02 + i*0.01,2)])

- Mol Bubi és Donkey Republic-ra keresek 1000 méteres körben
- a találatok bekerülnek egy-egy Pandas DataFrame-be
- majd CSV-be mentésre kerülnek

In [8]:
keyword_mol = 'Mol%20Bubi'
keyword_don = 'Donkey%20Republic'
radius = '1000'

for i in range(len(gps)):
    #print(f"========== {gps[i][0]}, {gps[i][1]} ==========")    
    mol = collect_data(keyword_mol, gps[i][0], gps[i][1], radius, api_key, mol)
    don = collect_data(keyword_don, gps[i][0], gps[i][1], radius, api_key, don)

In [9]:
mol

,StationName,PostalCode,City,District,Street,Latitude,Longitude
0,1127-Fraknó utca - Bánk Bán utca,1115,Budapest,11,Fraknó utca,47.468624,19.029269
1,1131-Kelenföld vasútállomás M (Etele tér),1115,Budapest,11,Somogyi út,47.464447,19.023036
2,1125-Bikás park M,1119,Budapest,11,Vahot utca,47.464936,19.033556
3,1122-Szent Gellért-templom,1115,Budapest,11,Somogyi út,47.46861,19.024456
4,1124-Vahot utca - Wartha Vince utca,1119,Budapest,11,Vahot utca,47.465117,19.027727
...,...,...,...,...,...,...,...
153,0815-Diószegi Sámuel utca - Elnök utca,1089,Budapest,8,Diószegi Sámuel utca,47.483218,19.091532
154,1322-Rákosrendező vasútállomás,1135,Budapest,13,Szegedi út,47.52551,19.088246
155,0910-Népliget M (buszpályaudvar),1091,Budapest,10,Üllői út,47.474918,19.099345
156,0913-Ecseri út M,1098,Budapest,9,Ecseri út,47.47109,19.10987


In [10]:
don

,StationName,PostalCode,City,District,Street,Latitude,Longitude
0,Etele Park,1119,Budapest,11,Etele út,47.464032,19.027674
1,Kelenföld Pu.,1115,Budapest,11,Etele tér,47.464213,19.022662
2,Bikás Park,1119,Budapest,11,Vahot utca,47.465073,19.033405
3,Bikás Park II,1115,Budapest,11,Bártfai utca,47.466858,19.030056
4,Bártfai u.,1119,Budapest,11,Etele út,47.464172,19.027554
...,...,...,...,...,...,...,...
237,Sárkány utca,1089,Budapest,8,Orczy út,47.485632,19.092679
238,II. János Pál pápa tér 22.,1081,Budapest,8,II. János Pál pápa tér,47.4979,19.079308
239,II. János Pál pápa tér - Metro,1081,Budapest,8,II. János Pál pápa tér,47.495271,19.077336
240,Hősök,1146,Budapest,14,None,47.514107,19.078049


In [11]:
mol.to_csv(r'C:\vremek\mol_bubi.csv', sep=';', index=False)
don.to_csv(r'C:\vremek\donkey_republic.csv', sep=';', index=False)